In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2

In [2]:
# Define paths
base_dir ="D:\Projects\dog breed"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
validation_dir = os.path.join(base_dir, "valid")

In [3]:
# Image dimensions and parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
NUM_CLASSES = 70 # Total number of dog breeds

In [4]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
 
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
 
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
 
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
 

Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [8]:
# Baseline model
def create_baseline_model(input_shape, num_classes):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model
 

In [9]:
# CNN Model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [22]:
# Pre-trained MobileNetV2 Model
def create_pretrained_model(input_shape, num_classes):
    # Load MobileNetV2 without the top classification layer
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model layers

    # Add custom classification layers
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # Reduces feature maps to a single vector
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [10]:
# Compile and train the model
def train_model(model, train_data, validation_data, epochs=25):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs,
        callbacks=[early_stopping]
    )
    return history

# Train baseline model

In [12]:
# Train baseline model
print("Training Baseline Model...")
baseline_model = create_baseline_model((IMG_HEIGHT, IMG_WIDTH, 3), NUM_CLASSES)
train_model(baseline_model, train_generator, validation_generator, epochs=10)

Training Baseline Model...
Epoch 1/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 438s 2s/step - accuracy: 0.0195 - loss: 21.9009 - val_accuracy: 0.0143 - val_loss: 4.2488
Epoch 2/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 233s 917ms/step - accuracy: 0.0258 - loss: 4.2521 - val_accuracy: 0.0143 - val_loss: 4.2498
Epoch 3/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 274s 1s/step - accuracy: 0.0276 - loss: 4.2403 - val_accuracy: 0.0143 - val_loss: 4.2511
Epoch 4/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 219s 864ms/step - accuracy: 0.0244 - loss: 4.2357 - val_accuracy: 0.0143 - val_loss: 4.2527
Epoch 5/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 218s 858ms/step - accuracy: 0.0266 - loss: 4.2333 - val_accuracy: 0.0143 - val_loss: 4.2544
Epoch 6/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 217s 855ms/step - accuracy: 0.0237 - loss: 4.2322 - val_accuracy: 0.0143 - val_loss: 4.2561


In [14]:
# Evaluate model on test data
test_loss, test_acc = baseline_model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 398ms/step - accuracy: 0.0019 - loss: 4.2513  
Test Accuracy: 0.01


# Train CNN model

In [16]:
# Train CNN model
print("Training CNN Model...")
cnn_model = create_cnn_model((IMG_HEIGHT, IMG_WIDTH, 3), NUM_CLASSES)
history = train_model(cnn_model, train_generator, validation_generator, epochs=25)

Training CNN Model...
Epoch 1/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 693s 3s/step - accuracy: 0.0206 - loss: 4.3201 - val_accuracy: 0.0400 - val_loss: 4.1828
Epoch 2/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 700s 3s/step - accuracy: 0.0355 - loss: 4.1393 - val_accuracy: 0.0543 - val_loss: 3.9698
Epoch 3/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 710s 3s/step - accuracy: 0.0403 - loss: 4.0375 - val_accuracy: 0.0429 - val_loss: 3.9219
Epoch 4/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 703s 3s/step - accuracy: 0.0466 - loss: 3.9896 - val_accuracy: 0.0586 - val_loss: 3.8186
Epoch 5/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 688s 3s/step - accuracy: 0.0510 - loss: 3.9245 - val_accuracy: 0.0800 - val_loss: 3.7589
Epoch 6/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 676s 3s/step - accuracy: 0.0593 - loss: 3.8671 - val_accuracy: 0.0814 - val_loss: 3.6803
Epoch 7/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 592s 2s/step - accuracy: 0.0618 - loss: 3.8518 - val_accuracy: 0.0957 - val_loss: 3.6750
Epoch 8/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 588s 2s/step - accuracy: 0.0674 - l

In [17]:
# Evaluate model on test data
test_loss, test_acc = cnn_model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 488ms/step - accuracy: 0.2263 - loss: 2.9522
Test Accuracy: 0.22


# Train using pre-trained model

In [25]:
# Train the pre-trained model
print("Training Pre-trained Model...")
pretrained_model = create_pretrained_model((IMG_HEIGHT, IMG_WIDTH, 3), NUM_CLASSES)
history = train_model(pretrained_model, train_generator, validation_generator, epochs=20)

Training Pre-trained Model...
Epoch 1/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 384s 1s/step - accuracy: 0.2731 - loss: 3.0657 - val_accuracy: 0.8729 - val_loss: 0.6331
Epoch 2/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 370s 1s/step - accuracy: 0.6784 - loss: 1.1203 - val_accuracy: 0.9029 - val_loss: 0.5549
Epoch 3/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.7304 - loss: 0.9182 - val_accuracy: 0.9114 - val_loss: 0.5327
Epoch 4/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.7596 - loss: 0.8053 - val_accuracy: 0.9014 - val_loss: 0.5416
Epoch 5/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 406s 2s/step - accuracy: 0.7623 - loss: 0.7594 - val_accuracy: 0.9057 - val_loss: 0.5609
Epoch 6/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 408s 2s/step - accuracy: 0.7938 - loss: 0.6806 - val_accuracy: 0.9100 - val_loss: 0.6178
Epoch 7/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - accuracy: 0.7927 - loss: 0.7018 - val_accuracy: 0.9143 - val_loss: 0.6112
Epoch 8/20
249/249 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.

In [30]:
# Evaluate model on test data
test_loss, test_acc = pretrained_model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.9477 - loss: 0.2003
Test Accuracy: 0.94


# ----------------------------------------------------------------------

In [31]:
# Classification report
y_true = test_generator.classes
y_pred = pretrained_model.predict(test_generator)
y_pred_classes = y_pred.argmax(axis=1)

22/22 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step


In [32]:
 print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys()))

Classification Report:
                   precision    recall  f1-score   support

           Afghan       1.00      1.00      1.00        10
 African Wild Dog       1.00      1.00      1.00        10
         Airedale       1.00      1.00      1.00        10
American Hairless       1.00      0.80      0.89        10
 American Spaniel       0.69      0.90      0.78        10
          Basenji       0.91      1.00      0.95        10
           Basset       0.91      1.00      0.95        10
           Beagle       1.00      0.90      0.95        10
   Bearded Collie       1.00      1.00      1.00        10
         Bermaise       1.00      1.00      1.00        10
     Bichon Frise       1.00      1.00      1.00        10
         Blenheim       1.00      1.00      1.00        10
       Bloodhound       1.00      1.00      1.00        10
         Bluetick       1.00      1.00      1.00        10
    Border Collie       1.00      1.00      1.00        10
           Borzoi       1.00    

In [34]:
pretrained_model.save("my_model.keras")